<a href="https://colab.research.google.com/github/anilsah895/skin_diseases_detection/blob/master/VGG19_REsearch_Using_a_pre_trained_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Base_dir="gdrive/My Drive/Skin_Data1/"#content/gdrive/My Drive/Colab Notebooks/Skin_Data1
train_path = Base_dir+"train"
validation_path = Base_dir+"validation"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
from keras import models
from keras import layers
from keras import optimizers
from keras.applications import VGG19
#local_weights_file = "gdrive/My Drive/skindata/vgg16_weights_tf_dim_ordering_tf_kernels.h5"

pre_trained_model = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

#pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False
  
# Print the model summary
pre_trained_model.summary()

Using TensorFlow backend.
W0711 01:57:11.723516 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 01:57:11.769267 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 01:57:11.780806 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 01:57:11.827428 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



80142336/80134624 [==============================] - 2s 0us/step


W0711 01:57:14.423285 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0711 01:57:14.424638 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [3]:
# Check the trainable status of the individual layers
for layer in pre_trained_model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f215e3504e0> False
<keras.layers.convolutional.Conv2D object at 0x7f2136b99668> False
<keras.layers.convolutional.Conv2D object at 0x7f2136b99518> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2136b99c18> False
<keras.layers.convolutional.Conv2D object at 0x7f2136bc3908> False
<keras.layers.convolutional.Conv2D object at 0x7f21363726d8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f2136372e48> False
<keras.layers.convolutional.Conv2D object at 0x7f2136388da0> False
<keras.layers.convolutional.Conv2D object at 0x7f21363342e8> False
<keras.layers.convolutional.Conv2D object at 0x7f2136334b70> False
<keras.layers.convolutional.Conv2D object at 0x7f21362e6b00> False
<keras.layers.pooling.MaxPooling2D object at 0x7f21362f95c0> False
<keras.layers.convolutional.Conv2D object at 0x7f21362f9898> False
<keras.layers.convolutional.Conv2D object at 0x7f21362a8c18> False
<keras.layers.convolutional.Conv2D object at 0x7f21362bfe80>

In [0]:
#conv_base.summary()
with open(Base_dir+"vgg19MODEL_SUMMARY_FILE.text","w") as fh:
    pre_trained_model.summary(print_fn=lambda line: fh.write(line + "\n"))

In [0]:
import tensorflow as tf
# Define a Callback class that stops training once accuracy reaches 98.85%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.9885):
      print("\nReached 98.85% accuracy so cancelling training!")
      self.model.stop_training = True

In [6]:
model = models.Sequential()
model.add(pre_trained_model)
#model.add(layers.Flatten())
#model.last_layer.output()
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
#pre_trained_model.trainable = False

model.summary()

W0711 01:57:48.099027 139781859743616 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 24,224,330
Trainable params: 4,199,946
Non-trainable params: 20,024,384
_________________________________________________________________


In [0]:
#from keras import models
from keras.preprocessing.image import ImageDataGenerator
callback1=myCallback()

model.compile(optimizer = optimizers.RMSprop(lr=0.00003, rho=0.9, epsilon=1e-08, decay=0.00001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
# Change the batchsize according to your system RAM
train_batchsize = 64
val_batchsize = 32
 
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(150, 150),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(150, 150),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=200,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,
callbacks=[callback1])

W0711 01:57:57.447946 139781859743616 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Found 4400 images belonging to 10 classes.


W0711 01:58:10.017091 139781859743616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Found 1100 images belonging to 10 classes.
Epoch 1/200
24/68 [=========>....................] - ETA: 14:08 - loss: 2.3241 - acc: 0.1302

In [0]:
model.save(Base_dir+'beforeVGG19skin_net100.hdf5')


In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy before fine tuning')
plt.legend(loc=0)
plt.savefig(Base_dir+"vgg19TRAINING_PLOT_FILE.png", bbox_inches='tight')
plt.savefig(Base_dir+"vgg19TRAINING_PLOT_FILE.png", bbox_inches='tight')

In [0]:
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss before fine tuning')
plt.legend()
plt.savefig(Base_dir+"vgg19VALIDATION_PLOT_FILE.png", bbox_inches='tight')
plt.savefig(Base_dir+"vgg19VALIDATION_PLOT_FILE.png", bbox_inches='tight')

In [0]:
model.summary()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy after fine tuning')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.models import load_model
model1 = load_model(Base_dir+'beforeVGG19skin_net100.hdf5')
validation_generator.reset()
Y_pred = model1.predict_generator(validation_generator, validation_generator.samples//validation_generator.batch_size+1)
y_pred = np.argmax(Y_pred,axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ["Acne-Rosacea", "Basal cell carcinoma"," Hair Loss Alopecia and other Hair Diseases",
             " Herpes"," Melanoma Skin Cancer Nevi and Moles",
            " Nail Fungus and other Nail Disease" , "Urticaria Hives"," Vasculitis"," Warts","seborrheic-keratoses"]
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [0]:
model.summary()

In [0]:
pre_trained_model.trainable = True

set_trainable = False
for layer in pre_trained_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False



In [0]:
model.compile(optimizer = optimizers.RMSprop(lr=0.000003, rho=0.9, epsilon=1e-08, decay=0.00001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [0]:
model.summary()

In [0]:
callback1=myCallback()
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=200,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,
callbacks=[callback1])

In [0]:
model.save(Base_dir+'VGG19skin_net_after_fineTuningAF.hdf5')


In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy after fine Tuning')
plt.legend(loc=0)
plt.savefig(Base_dir+"vgg19TRAINING_PLOT_FILE_after_fineTune.png", bbox_inches='tight')

In [0]:
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss after fine Tuning')
plt.legend()
plt.savefig(Base_dir+"vgg19VALIDATION_PLOT_FILE_with_finetune.png", bbox_inches='tight')

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy after fine tuning')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss after fine tuning')
plt.legend()

plt.show()

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.models import load_model
#content/gdrive/My Drive/Skin_Data1/weight1
model = load_model(Base_dir+'VGG19skin_net_after_fineTuningAF.hdf5')
Y_pred = model.predict_generator(validation_generator, validation_generator.samples//validation_generator.batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ["Acne-Rosacea", "Basal cell carcinoma"," Hair Loss Alopecia and other Hair Diseases",
             " Herpes"," Melanoma Skin Cancer Nevi and Moles",
            " Nail Fungus and other Nail Disease" , "Urticaria Hives"," Vasculitis"," Warts","seborrheic-keratoses"]
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [0]:
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
%matplotlib inline
plot_model(pre_trained_model, to_file=Base_dir+'model.png')
SVG(model_to_dot(pre_trained_model).create(prog='dot', format='svg'))


In [0]:
from keras.models import load_model
from google.colab import files
from keras.preprocessing import image
import numpy as np
from matplotlib.pyplot import imshow
from keras.applications.imagenet_utils import preprocess_input
import scipy.misc

model = load_model(Base_dir+'VGG19skin_net_after_fineTuningAF.hdf5')
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
CATEGORIES =["Acne-Rosacea", "Basal cell carcinoma"," Hair Loss Alopecia and other Hair Diseases",
             " Herpes"," Melanoma Skin Cancer Nevi and Moles",
            " Nail Fungus and other Nail Disease" , "Urticaria Hives"," Vasculitis"," Warts","seborrheic-keratoses"]
uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  #print('Input image shape:', x.shape)
  
  images = np.vstack([x])
  classes = model.predict(images)#, batch_size=10)
  #print(fn)
  
  print(classes[0])
  maximum=0.9
  for i,value in enumerate(classes[0]):
    if value>maximum:
      index=i
  print("The predicted disease found to be "+CATEGORIES[index]) 
  #print(type(CATEGORIES[int(classes[0][0])]))  
  
 